In [ ]:
from sklift.datasets import fetch_hillstrom
from sklearn.datasets import fetch_openml  # or from dirty_cat.datasets if using dirty_cat

import pandas as pd

In [ ]:
# Fetch the Hillstrom dataset with the target column specified
df_bunch = fetch_hillstrom(target_col='visit')
df = pd.DataFrame(df_bunch.data, columns=df_bunch.feature_names)
df['target'] = df_bunch.target
df['treatment'] = df_bunch.treatment
df.head()


,recency,history_segment,history,mens,womens,zip_code,newbie,channel,target
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,0


In [15]:
dataset = fetch_hillstrom(target_col='visit')
data, target, treatment = dataset.data, dataset.target, dataset.treatment

In [17]:
treatment

0        Womens E-Mail
1            No E-Mail
2        Womens E-Mail
3          Mens E-Mail
4        Womens E-Mail
             ...      
63995      Mens E-Mail
63996      Mens E-Mail
63997      Mens E-Mail
63998    Womens E-Mail
63999      Mens E-Mail
Name: segment, Length: 64000, dtype: object

In [14]:
df_bunch.feature_names

['recency',
 'history_segment',
 'history',
 'mens',
 'womens',
 'zip_code',
 'newbie',
 'channel']